In [120]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

In [121]:
Cust = pd.read_csv('lv2-2401.csv')

In [122]:
Cust.shape

(2000, 9)

# 1.

## 1-1.

In [123]:
df1 = Cust.copy()

In [124]:
def get_age_p(x):
    if x < 20:
        return '10s'
    elif x < 30:
        return '20s'
    elif x < 40:
        return '30s'
    elif x < 50:
        return '40s'
    elif x < 60:
        return '50s'
    else:
        return 'other'

In [125]:
df1['AGE_P'] = df1['AGE'].map(get_age_p)

## 1-2.

In [126]:
Agg24 = df1.groupby('AGE_P')\
            .agg({'SPENDING': 'mean', 'C_ID': 'count'})\
            .rename(columns={'SPENDING': 'AVG_SPEND', 'C_ID': 'CUST_CNT'})

## 1-3.

In [127]:
Agg24.loc[Agg24['AVG_SPEND'] == Agg24['AVG_SPEND'].max(), 'CUST_CNT']

AGE_P
10s    380
Name: CUST_CNT, dtype: int64

# 2.

## 2-1.

In [128]:
df2 = Cust.copy()

In [129]:
df2.columns
cols = ['AGE', 'INCOME', 'WORK_EXP', 'FAMILY', 'SPENDING']
cols

['AGE', 'INCOME', 'WORK_EXP', 'FAMILY', 'SPENDING']

In [130]:
from sklearn.preprocessing import StandardScaler

In [131]:
scaler = StandardScaler().fit(X=df2[cols])

In [132]:
df2[cols] = pd.DataFrame(scaler.transform(df2[cols]), columns=cols)
df2.head()

,C_ID,GENDER,AGE,INCOME,JOB,WORK_EXP,FAMILY,SPENDING,GRADE
0,1,Male,-1.098933,-2.093501,B,-0.791207,0.117497,-0.428339,prestige
1,2,Male,-0.996920,-1.656133,D,-0.281162,-0.390051,1.075546,royal blue
2,3,Female,-1.047927,-0.540845,D,-0.791207,-1.405148,-1.609962,general
3,4,Female,-0.894907,-1.131292,G,-1.046230,-0.897599,0.932319,royal blue
4,5,Female,-0.486856,-1.590528,C,-0.536185,1.132594,-0.392532,prestige


## 2-2.
## 2-3.

In [133]:
df2 = df2.sort_values(by='C_ID')

In [134]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

- cell merge : shift + m

In [135]:
result = pd.DataFrame()

for i in [3, 4]:
    model = KMeans(n_clusters=i, random_state=1234, init='random').fit(df2[cols])
    clusters = model.predict(df2[cols])
    score = silhouette_score(df2[cols], clusters)
    
    result = result.append({'cluster': i, 'score': score}, ignore_index=True)

In [136]:
result['score'].max().round(3)

0.158

# 3.

## 3-1.

In [138]:
df3 = Cust[~(Cust['AGE'] < 14) & ~(Cust['JOB'].isna())]
df3.shape

(1714, 9)

## 3-2.

In [139]:
df3 = pd.get_dummies(df3, columns=['GENDER', 'JOB'], drop_first=True)

## 3-3.

In [140]:
TrainSet6 = df3[df3['C_ID'] % 3 != 0]
TestSet6 = df3[df3['C_ID'] % 3 == 0]
display(TrainSet6.shape, TestSet6.shape)

(1148, 16)

(566, 16)

## 3-4.

In [141]:
TrainSet6.columns
cols = ['AGE', 'INCOME', 'WORK_EXP', 'FAMILY',
       'GENDER_Male', 'JOB_B', 'JOB_C', 'JOB_D', 'JOB_E', 'JOB_F', 'JOB_G',
       'JOB_H', 'JOB_I']
len(cols)

13

In [142]:
from sklearn.tree import DecisionTreeClassifier

In [143]:
model = DecisionTreeClassifier(max_depth=5, random_state=1234).fit(X=TrainSet6[cols], y=TrainSet6['GRADE'])

In [144]:
pred = model.predict(X=TestSet6[cols])

## 3-5.

In [145]:
from sklearn.metrics import accuracy_score

In [148]:
round(accuracy_score(y_true=TestSet6['GRADE'], y_pred=pred) * 100, 2)

33.22